In [1]:
from sqlalchemy import create_engine, MetaData, Column, Table, text, inspect
from sqlalchemy.types import Integer, Text, String
import sqlite3
import pandas as pd

C:\Users\dark1n\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\dark1n\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\dark1n\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# cria e inicia a conexão com ambos os bancos, é na connection string que conseguimos passar os parâmetros de login
# exemplo para mysql: mysql://{user}:{password]@{ip}/{dbname}

source_engine = create_engine("sqlite:///source.db")
target_engine = create_engine("sqlite:///target.db")

source_conn = source_engine.connect()
target_conn = target_engine.connect()

In [3]:
# popula dados no banco inicial

metadata_obj = MetaData()

users = Table(
    'users',
    metadata_obj,
    Column("id", Integer, primary_key=True, autoincrement=True),
    Column("name", String(60), nullable=False, key="name")
)

if not inspect(source_engine).has_table("users"):
    users.create(source_engine)


data = [
    {'name': 'João'},
    {'name': 'Maria'},
    {'name': 'André'}
]

source_conn.execute(users.insert(), data)

pd.read_sql("select * from users", source_conn)


,id,name
0,1,João
1,2,Maria
2,3,André


In [4]:
#reflete os metadados da engine fonte com a engine target e cria-os no target
metadata_obj.create_all(target_engine)

In [7]:
#lê as informações da tabela inicial usando um chunksize e insere os valores na tabela destino

for df in pd.read_sql('SELECT * FROM users', con=source_conn, chunksize=1000):
    df.to_sql(
        con=target_conn,
        name='users',
        index=False,
        if_exists='replace'
    )


pd.read_sql("select * from users", target_conn)

,id,name
0,1,João
1,2,Maria
2,3,André


In [13]:
#tudo junto em uma classe!

class dbsync:

    def __init__(self, source_engine, target_engine, table_schema, chunksize, if_exists):
        self.source_engine = source_engine
        self.target_engine = target_engine
        self.table_schema = table_schema
        self.chunksize = chunksize
        self.if_exists = if_exists
        self.source_conn = source_engine.connect()
        self.target_conn = target_engine.connect()

    def run(self):
        if not inspect(source_engine).has_table(self.table_schema.name):
            raise ValueError('table not found on source db')

        if not inspect(target_engine).has_table(self.table_schema.name):
            self.table_schema.create(bind = target_conn)

        for df in pd.read_sql(f"SELECT * FROM {self.table_schema.name}", con=source_conn, chunksize=self.chunksize):
            df.to_sql(
                con=target_conn,
                name= self.table_schema.name,
                index=False,
                if_exists= self.if_exists
            )        


In [14]:
metadata_obj = MetaData()

users = Table(
    'users',
    metadata_obj,
    Column("id", Integer, primary_key=True, autoincrement=True),
    Column("name", String(60), nullable=False, key="name")
)

sync = dbsync(source_engine = source_engine, target_engine = target_engine, table_schema = users, chunksize=1024, if_exists='replace')
sync.run()


In [15]:
pd.read_sql("select * from users", target_conn)

,id,name
0,1,João
1,2,Maria
2,3,André
